In [1]:
import os
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np
import cv2 as cv
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [2]:
# Replace 'folder_path' with the actual path to your folder
folder_path = r'D:\motion_blur_dataset\test_set'

# Get a list of all subfolder names in the folder
subfolder_names = [f for f in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, f))]
for index,name in enumerate(subfolder_names):
    subfolder_names[index] = name[4:]

In [3]:
print(len(subfolder_names))

10


In [4]:
# for index,path in enumerate(subfolder_names):
#     standard_path = f"D:\motion_blur_dataset\scans\sub-{path}\\anat\sub-{path}_acq-standard_T1w.nii.gz"
#     motion_1_path = f"D:\motion_blur_dataset\scans\sub-{path}\\anat\sub-{path}_acq-headmotion1_T1w.nii.gz"
#     motion_2_path = f"D:\motion_blur_dataset\scans\sub-{path}\\anat\sub-{path}_acq-headmotion2_T1w.nii.gz"
#     slice_number = 175

#     try:
#         standard_image = nib.load(standard_path).get_fdata()
#         standard_image = scaler.fit_transform(standard_image.reshape(-1, standard_image.shape[-1])).reshape(standard_image.shape) #convert to a range of [0,1]
#         standard_image = standard_image[:,:,slice_number]
#         # standard_image = cv.resize(standard_image, (256,256))
#         # standard_image = np.stack((standard_image,) * 3, axis=-1)

#         motion_1_image = nib.load(motion_1_path).get_fdata()
#         motion_1_image = scaler.fit_transform(motion_1_image.reshape(-1, motion_1_image.shape[-1])).reshape(motion_1_image.shape) #convert to a range of [0,1]
#         motion_1_image = motion_1_image[:,:,slice_number]
#         # motion_1_image = cv.resize(motion_1_image, (256,256))

#         motion_2_image = nib.load(motion_2_path).get_fdata()
#         motion_2_image = scaler.fit_transform(motion_2_image.reshape(-1, motion_2_image.shape[-1])).reshape(motion_2_image.shape) #convert to a range of [0,1]
#         motion_2_image = motion_2_image[:,:,slice_number]
#         # motion_2_image = cv.resize(motion_2_image, (256,256))
#     except:
#         continue
    
#     # os.mkdir(f"F:\FYP\FYP\motion_blur_repo\png_scans\scan_{path}")
#     # cv.imwrite(f'F:\FYP\FYP\motion_blur_repo\png_scans\scan_{path}\standard_{path}_png.png', standard_image*255)
#     # cv.imwrite(f"F:\FYP\FYP\motion_blur_repo\png_scans\scan_{path}\motion1_{path}_png.png",motion_1_image*255)
#     # cv.imwrite(f"F:\FYP\FYP\motion_blur_repo\png_scans\scan_{path}\motion2_{path}_png.png",motion_2_image*255)
#     # np.save(f"F:/FYP/deblur/deblur_data/delur_Y_data/slice_{slice_number}_{path}_standard.npy",standard_image)
#     # np.save(f"F:/FYP/deblur/deblur_data/motion_1_X_data/slice{slice_number}_{path}_motion_1.npy",motion_1_image)
#     # np.save(f"F:/FYP/deblur/deblur_data/motion_2_X_data/slice{slice_number}_{path}_motion_2.npy",motion_2_image)
#     # cv.imwrite(f"C:/Users/Milan/Desktop/test_dataset/motion1_{path}_png.png",standard_image*255)

#     # Create a figure
#     fig = plt.figure(figsize=(30, 30))

#     # Create subplots and store their references
#     ax1 = plt.subplot(121)
#     # ax2 = plt.subplot(132)
#     ax3 = plt.subplot(122)

#     # Add images and titles to each subplot
#     ax1.imshow(standard_image, cmap="gray")
#     ax1.set_title("Scan without motion artifacts")

#     # ax2.imshow(motion_1_image, cmap="gray")
#     # ax2.set_title("Scan under motion condition 1")

#     ax3.imshow(motion_2_image, cmap="gray")
#     ax3.set_title("Scan with motion artifacts")

#     # Adjust spacing between subplots
#     # plt.tight_layout()
#     # plt.savefig(f"F:\FYP\FYP\motion_blur_images\scan_{path}.png")
#     # Display the figure
#     # plt.show()
#     if index >10:
#         break

# Creating motion artifacts

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft2, fftshift,ifft2, ifftshift
import cv2

def add_rotation(angle, image):
    # Define the rotation angle (in degrees)

    # Get the center of the image
    height, width = image.shape[:2]
    center = (width / 2, height / 2)

    # Define the rotation matrix
    rotation_matrix = cv.getRotationMatrix2D(center, angle, 1.0)

    # Apply the rotation to the image
    rotated_image = cv.warpAffine(image, rotation_matrix, (width, height))

    return rotated_image

def add_translation(shift_x, shift_y, image):
    # Define the rotation angle (in degrees)

    height, width = image.shape[:2]

    # Define the translation matrix
    translation_matrix = np.float32([[1, 0, shift_x], [0, 1, shift_y]])

    # Apply the translation to the rotated image
    shifted_image = cv2.warpAffine(image, translation_matrix, (width, height))

    return shifted_image

def fourier_transform(image):
    # print("here in FT")
    if len(image.shape) == 3:
        image = np.mean(image, axis=-1)

    # Perform 2D Fourier Transform
    fft_result = fft2(image)

    # Shift zero frequency components to the center
    fft_result_shifted = fftshift(fft_result)

    # Calculate the magnitude spectrum (log scale for better visualization)
    magnitude_spectrum = np.log(np.abs(fft_result_shifted) + 1)

    return magnitude_spectrum,fft_result_shifted

def inverse_fourier(fft_result_shifted):
    # print("Here in IFT")
    # Inverse shift to prepare for inverse Fourier transform
    fft_result_unshifted_masked = ifftshift(fft_result_shifted)
    # Inverse Fourier Transform
    original_filtered = np.abs(ifft2(fft_result_unshifted_masked))
    return original_filtered

import random
def weighted_random():
    # Generate a random number between 0 and 1
    if random.random() < 0.1:
        # 10% of the time, return a number between 6 and 10
        return random.randint(6, 10)
    else:
        # 90% of the time, return a number between 0 and 5
        return random.randint(0, 5)


In [6]:
# slice_number = 175
# for index,path in enumerate(subfolder_names):
    
#     standard_path = f"D:\motion_blur_dataset\scans\sub-{path}\\anat\sub-{path}_acq-standard_T1w.nii.gz"
#     motion_1_path = f"D:\motion_blur_dataset\scans\sub-{path}\\anat\sub-{path}_acq-headmotion1_T1w.nii.gz"
#     motion_2_path = f"D:\motion_blur_dataset\scans\sub-{path}\\anat\sub-{path}_acq-headmotion2_T1w.nii.gz"
    

#     try:
        
#         standard_image = nib.load(standard_path).get_fdata()
#         standard_image = scaler.fit_transform(standard_image.reshape(-1, standard_image.shape[-1])).reshape(standard_image.shape) #convert to a range of [0,1]
#         standard_image = standard_image[:,:,slice_number]
#         standard_image = cv.resize(standard_image, (256,256))
#         angle = 3
#         rotated_image = add_rotation(angle,standard_image)

#         magnitude_spectrum,fft_result_shifted = fourier_transform(standard_image)
#         rotated_magnitude_spectrum,rotated_fft_result_shifted = fourier_transform(rotated_image)

#         number_of_lines = 50
#         line_indexes = np.linspace(0, rotated_image.shape[1]-2, number_of_lines).astype(int)

#         motion_corrupted = rotated_fft_result_shifted
#         for i in line_indexes:
#             extracted = fft_result_shifted[:,i:i+1]
#             motion_corrupted[:,i:i+1] = extracted

#         inverse_fourier_corrupted = inverse_fourier(motion_corrupted)
#         inverse_fourier_corrupted = add_rotation(-angle,inverse_fourier_corrupted)

#         plt.imsave(f"C:/Users/Milan/Desktop/artifact_y/{slice_number}_{path}_acq-standard_T1w.png",standard_image,cmap="gray")
#         plt.imsave(f"C:/Users/Milan/Desktop/artifact_x/{slice_number}_{path}_acq-standard_T1w_corrupted.png",inverse_fourier_corrupted,cmap="gray")
#         # print("here",index,f"C:/Users/Milan/Desktop/artifact_y/{slice_number}_{path}_acq-standard_T1w.png")

#     except:
#         print("here 2")
#         continue

In [7]:
for slice_number in range(150,186,1):
    for index,path in enumerate(subfolder_names):
        print(f"Slice number-{slice_number} scan number - {index}")
        standard_path = f"D:\motion_blur_dataset\\test_set\sub-{path}\\anat\sub-{path}_acq-standard_T1w.nii.gz"
        motion_1_path = f"D:\motion_blur_dataset\scans\sub-{path}\\anat\sub-{path}_acq-headmotion1_T1w.nii.gz"
        motion_2_path = f"D:\motion_blur_dataset\scans\sub-{path}\\anat\sub-{path}_acq-headmotion2_T1w.nii.gz"
        
        try: 
            standard_image = nib.load(standard_path).get_fdata()
            standard_image = scaler.fit_transform(standard_image.reshape(-1, standard_image.shape[-1])).reshape(standard_image.shape) #convert to a range of [0,1]
            standard_image = standard_image[:,:,slice_number]
            standard_image = np.pad(standard_image, ((32, 32), (0, 0)), mode='constant', constant_values=0)
        except:
            print(f"Error with file {standard_path}")
            continue
        standard_image = cv2.rotate(standard_image, cv2.ROTATE_90_COUNTERCLOCKWISE)
        
        angle = 7
        # rotated_image = add_rotation(angle,standard_image)
        rotated_image = add_translation(1,0,standard_image)

        magnitude_spectrum,fft_result_shifted = fourier_transform(standard_image)
        rotated_magnitude_spectrum,rotated_fft_result_shifted = fourier_transform(rotated_image)
        motion_corrupted = fft_result_shifted

        number_of_lines = 100 # Define the number of lines to be Sampled from 
        width = 1
        line_indexes = np.random.randint(0, rotated_image.shape[1]-2, number_of_lines)

        for i in line_indexes:
            extracted = rotated_fft_result_shifted[:,i:i+width]
            motion_corrupted[:,i:i+width] = extracted
            # extracted = rotated_fft_result_shifted[i:i+width,:]
            # motion_corrupted[i:i+width,:] = extracted

        inverse_fourier_corrupted = inverse_fourier(motion_corrupted)
        # inverse_fourier_corrupted = add_rotation(-angle,inverse_fourier_corrupted)

        # standard_image = cv.resize(standard_image, (256,256))
        # inverse_fourier_corrupted = cv.resize(inverse_fourier_corrupted, (256,256))

        # random_integer = weighted_random()
            
        # if (random_integer>5):
        #     plt.imsave(f"F:/FYP/FYP/MRART_val_data/Original/{slice_number}_{path}_acq-standard_motion_1_T1w.png",standard_image,cmap="gray")
        #     plt.imsave(f"F:/FYP/FYP/MRART_val_data/Corrupted/{slice_number}_{path}_acq-standard_T1w_motion_1_corrupted.png",inverse_fourier_corrupted,cmap="gray")
        # else:
        #     plt.imsave(f"F:/FYP/FYP/MRART_training_data/Original/{slice_number}_{path}_acq-standard_motion_1_T1w.png",standard_image,cmap="gray")
        #     plt.imsave(f"F:/FYP/FYP/MRART_training_data/Corrupted/{slice_number}_{path}_acq-standard_T1w_motion_1_corrupted.png",inverse_fourier_corrupted,cmap="gray")

        
        plt.imsave(f"C:/Users/Milan/Desktop/MRART_training_data/Original/{slice_number}_{path}_acq-standard_motion_1_T1w.png",standard_image,cmap="gray")
        plt.imsave(f"C:/Users/Milan\Desktop/MRART_training_data/Corrupted/{slice_number}_{path}_acq-standard_T1w_motion_1_corrupted.png",inverse_fourier_corrupted,cmap="gray")


Slice number-150 scan number - 0
Slice number-150 scan number - 1
Slice number-150 scan number - 2
Slice number-150 scan number - 3
Slice number-150 scan number - 4
Slice number-150 scan number - 5
Slice number-150 scan number - 6
Slice number-150 scan number - 7
Slice number-150 scan number - 8
Slice number-150 scan number - 9
Slice number-151 scan number - 0
Slice number-151 scan number - 1
Slice number-151 scan number - 2
Slice number-151 scan number - 3
Slice number-151 scan number - 4
Slice number-151 scan number - 5
Slice number-151 scan number - 6
Slice number-151 scan number - 7
Slice number-151 scan number - 8
Slice number-151 scan number - 9
Slice number-152 scan number - 0
Slice number-152 scan number - 1
Slice number-152 scan number - 2
Slice number-152 scan number - 3
Slice number-152 scan number - 4
Slice number-152 scan number - 5
Slice number-152 scan number - 6
Slice number-152 scan number - 7
Slice number-152 scan number - 8
Slice number-152 scan number - 9
Slice numb